In [5]:
import pandas as pd
from factor_analyzer import FactorAnalyzer
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# 데이터 가져오기
internalMetrics_df = pd.read_csv('result_rdb_internal_4.csv')

In [11]:
# 확인
internalMetrics_df.head()

,redis_version,redis_git_sha1,redis_git_dirty,redis_build_id,redis_mode,os,arch_bits,multiplexing_api,atomicvar_api,gcc_version,...,repl_backlog_first_byte_offset,repl_backlog_histlen,used_cpu_sys,used_cpu_user,used_cpu_sys_children,used_cpu_user_children,cluster_enabled,db0:keys,expires,avg_ttl
0,5.0.2,0,0,24934926e99aa4a4,standalone,Linux 4.15.0-1098-gcp x86_64,64,epoll,atomic-builtin,5.4.0,...,0,0,21.092774,6.039113,0.0,0.0,0,249666,0,0
1,5.0.2,0,0,24934926e99aa4a4,standalone,Linux 4.15.0-1098-gcp x86_64,64,epoll,atomic-builtin,5.4.0,...,0,0,20.643600,6.038766,0.0,0.0,0,249666,0,0
2,5.0.2,0,0,24934926e99aa4a4,standalone,Linux 4.15.0-1098-gcp x86_64,64,epoll,atomic-builtin,5.4.0,...,0,0,20.923632,5.806548,0.0,0.0,0,249666,0,0
3,5.0.2,0,0,24934926e99aa4a4,standalone,Linux 4.15.0-1098-gcp x86_64,64,epoll,atomic-builtin,5.4.0,...,0,0,22.507727,6.583683,0.0,0.0,0,249666,0,0
4,5.0.2,0,0,24934926e99aa4a4,standalone,Linux 4.15.0-1098-gcp x86_64,64,epoll,atomic-builtin,5.4.0,...,0,0,20.708273,5.782973,0.0,0.0,0,249666,0,0


## 변수 선택
### 버전 환경 관련
* redis_version
* redis_builid_id
* os
* gcc_version
* Executable
* config_file
* tcp_port
* run_id
* process_id

In [22]:
# 버전, 환경 관련 변수
var1_noRel = [
    'redis_version', 
    'redis_build_id', 
    'os',
    'gcc_version',
    'executable',
    'config_file',
    'tcp_port',
    'run_id',
    'process_id'
]

In [23]:
# 버전, 환경 관련 변수 확인
var1_noRel

['redis_version',
 'redis_build_id',
 'os',
 'gcc_version',
 'executable',
 'config_file',
 'tcp_port',
 'run_id',
 'process_id']

In [24]:
# 변수 제거 
internalMetrics_df = internalMetrics_df.drop(columns = var1_noRel, axis=0)

In [26]:
# 변수 제거 데이터프레임 확인
internalMetrics_df.head()

,redis_git_sha1,redis_git_dirty,redis_mode,arch_bits,multiplexing_api,atomicvar_api,uptime_in_seconds,uptime_in_days,hz,configured_hz,...,repl_backlog_first_byte_offset,repl_backlog_histlen,used_cpu_sys,used_cpu_user,used_cpu_sys_children,used_cpu_user_children,cluster_enabled,db0:keys,expires,avg_ttl
0,0,0,standalone,64,epoll,atomic-builtin,38,0,30,30,...,0,0,21.092774,6.039113,0.0,0.0,0,249666,0,0
1,0,0,standalone,64,epoll,atomic-builtin,37,0,18,18,...,0,0,20.643600,6.038766,0.0,0.0,0,249666,0,0
2,0,0,standalone,64,epoll,atomic-builtin,38,0,39,39,...,0,0,20.923632,5.806548,0.0,0.0,0,249666,0,0
3,0,0,standalone,64,epoll,atomic-builtin,42,0,6,6,...,0,0,22.507727,6.583683,0.0,0.0,0,249666,0,0
4,0,0,standalone,64,epoll,atomic-builtin,38,0,35,35,...,0,0,20.708273,5.782973,0.0,0.0,0,249666,0,0


### 모두 동일한 값
* 0 값
* 0이 아닌 값

In [34]:
for metric in internalMetrics_df:
    print(internalMetrics_df[metric].unique)

<bound method Series.unique of 0       0
1       0
2       0
3       0
4       0
       ..
9995    0
9996    0
9997    0
9998    0
9999    0
Name: redis_git_sha1, Length: 10000, dtype: int64>
<bound method Series.unique of 0       0
1       0
2       0
3       0
4       0
       ..
9995    0
9996    0
9997    0
9998    0
9999    0
Name: redis_git_dirty, Length: 10000, dtype: int64>
<bound method Series.unique of 0       standalone
1       standalone
2       standalone
3       standalone
4       standalone
           ...    
9995    standalone
9996    standalone
9997    standalone
9998    standalone
9999    standalone
Name: redis_mode, Length: 10000, dtype: object>
<bound method Series.unique of 0       64
1       64
2       64
3       64
4       64
        ..
9995    64
9996    64
9997    64
9998    64
9999    64
Name: arch_bits, Length: 10000, dtype: int64>
<bound method Series.unique of 0       epoll
1       epoll
2       epoll
3       epoll
4       epoll
        ...  
9995    epoll


### 범주형 변수 처리

In [ ]:
import pandas as pd

In [49]:
internalMetrics_df = pd.get_dummies(internalMetrics_df)
internalMetrics_df.head()

,redis_git_sha1,redis_git_dirty,arch_bits,uptime_in_seconds,uptime_in_days,hz,configured_hz,lru_clock,connected_clients,client_recent_max_input_buffer,...,master_replid_ffbbf40751005b2958ab7907bbd85be7a78f8ba4,master_replid_ffc54a7d649a0dd01aaf30a17d5dbdb0ed975703,master_replid_ffdbd26f71a7780b7b02534f923d701360171f0a,master_replid_ffe4703ce3afe5aa4fcbd3b209b1073c1cb160b0,master_replid_ffe9857611d79433bf3a6087b73511a64cbe9213,master_replid_ffea00f0a0f7e899f16af886821bea20ad9f5e8c,master_replid_ffeeb0aeec459a42b93fe97007b25554538a81cf,master_replid_fff782e367eaf4ee5dd0a119755f94e30219defe,master_replid_fff88ef9e52ef9a3ba6ac53d27fe0c6d1ac99a22,master_replid_fffba1787f48a6d6fb912a46e26bbb9f0f63e00e
0,0,0,64,38,0,30,30,10269216,1,2,...,0,0,0,0,0,0,0,0,0,0
1,0,0,64,37,0,18,18,10269260,1,2,...,0,0,0,0,0,0,0,0,0,0
2,0,0,64,38,0,39,39,10269304,1,2,...,0,0,0,0,0,0,0,0,0,0
3,0,0,64,42,0,6,6,10269353,1,2,...,0,0,0,0,0,0,0,0,0,0
4,0,0,64,38,0,35,35,10269397,1,2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,0,64,40,0,35,35,10713372,1,2,...,0,0,0,0,0,0,0,0,0,0
9996,0,0,64,40,0,2,2,10713418,1,2,...,0,0,0,0,0,0,0,0,0,0
9997,0,0,64,38,0,25,25,10713463,1,2,...,0,0,0,0,0,0,0,0,0,0
9998,0,0,64,40,0,11,11,10713510,1,2,...,0,0,0,0,0,0,0,0,0,0


## Factor Analysis

In [36]:
fa = FactorAnalyzer(n_factors=3, rotation='varimax')
fa.fit(internalMetrics_df)

ValueError: could not convert string to float: 'standalone'